<a href="https://colab.research.google.com/github/barkamoljon/Resume_Parser/blob/main/(From_Sratch)Resume_Parser_with_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Firstly, you must turn on GPU

In [ ]:
#do not install not used libraries

In [ ]:
!pip install -U spacy
!pip install spacy_transformers
!pip install gradio
!pip install PyPDF2==2.9.0
!pip install fitz
!pip install pip install PyMuPDF==1.19.0

In [ ]:
import spacy
import fitz
import gradio as gr
import json  #do not import if you do not use it
import PyPDF2
from spacy.tokens import DocBin #do not import if you do not use it
from tqdm import tqdm #do not import if you do not use it
from sklearn.model_selection import train_test_split #do not import if you do not use it

In [ ]:
cv_data = json.load(open('/content/data.txt', 'r'))

In [ ]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()
    
    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']
        
        ents = []
        entity_indices = []
        
        for start, end, label in annot:
            skip_entity = False
            for idx in entity_indices:
                skip_entity = True
                break
            if skip_entity == True:
                continue
            
            entity_indices = entity_indices + list(range(start,end))
        
            try:
                span = doc.char_span(start, end, label=label, aligment_model='strict')
            except:
                continue
            if span is None:
                err_data = str([start,end]) + "  " + str(text) + '\n'
                file.write(err_data)
            
            else:
                ents.append(span)
            
        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass
        
    return db

In [ ]:
train, test = train_test_split(cv_data, test_size=0.3)

In [ ]:
len(train), len(test)

In [ ]:
file = open('/content/drive/MyDrive/ResumeParser/model/train_file.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/ResumeParser/model/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('/content/drive/MyDrive/ResumeParser/model/test_data.spacy')

file.close()

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!python -m spacy train /content/drive/MyDrive/ResumeParser/config/config.cfg --output /content/drive/MyDrive/ResumeParser/model/output  --paths.train /content/drive/MyDrive/ResumeParser/model/train_data.spacy --paths.dev /content/drive/MyDrive/ResumeParser/model/test_data.spacy --gpu-id 0

In [ ]:
nlp = spacy.load('/content/drive/MyDrive/ResumeParser/model/output/model-best')

In [ ]:
def resume_parser(file_path):
  doc = fitz.open(file_path.name)
  text = ""
  for page in doc:
    text = text + str(page.get_text())
  doc = nlp(text)
  entity_dict = {}
  for ent in doc.ents:
    if ent.label_ not in entity_dict:
      entity_dict[ent.label_] = [ent.text]
    else:
      entity_dict[ent.label_].append(ent.text)
  entity_list = []
  for label, values in entity_dict.items():
    if len(values) == 1:
      entity_list.append({label: values[0]})
    else:
      entity_list.append({label: [value for value in values]})
  return entity_list

In [ ]:
resume_parser('/content/drive/MyDrive/ResumeParser/test/Resume Barkamol Urinboev.pdf')

In [ ]:
demo = gr.Interface(resume_parser, "file", "json")
demo.launch(debug=True,share=True)